In [126]:
import keras
import tensorflow as tf
from keras.applications import VGG16
from keras.utils.vis_utils import plot_model
import os, shutil
from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras import models
from keras.preprocessing import image
import numpy as np
import sklearn.metrics
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import itertools
import seaborn
print(keras.__version__)
print(tf.__version__)

2.4.3
2.4.1


In [15]:
pip install tf-nightly

     |████████████████████████████████| 408.4 MB 6.8 kB/s 
     |████████████████████████████████| 5.9 MB 7.3 MB/s 
     |████████████████████████████████| 462 kB 9.1 MB/s 
     |████████████████████████████████| 4.0 MB 3.8 MB/s 
     |████████████████████████████████| 4.4 MB 13.2 MB/s 
     |████████████████████████████████| 3.8 MB 3.5 MB/s 
ERROR: tensorflow 2.4.1 has requirement gast==0.3.3, but you'll have gast 0.4.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement grpcio~=1.32.0, but you'll have grpcio 1.34.1 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement h5py~=2.10.0, but you'll have h5py 3.1.0 which is incompatible.
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.32.0
    Uninstalling grpcio-1.32.0:
      Successfully uninstalled grpcio-1.32.0
  Attempting uninstall: gast
    Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Attempting uninstall: h5py
    Found e

In [134]:
# from tensorflow.keras.applications import EfficientNetB0
from keras.applications.vgg19 import VGG19


In [135]:
conv_base = VGG19(weights='imagenet',include_top=False,input_shape=(326, 326, 3), classifier_activation = 'softmax')
# conv_base = EfficientNetB0(
#     include_top=False, weights='imagenet', input_tensor=None,
#     input_shape=(350, 350, 3),
#     classifier_activation='softmax'
# )


80142336/80134624 [==============================] - 8s 0us/step


In [136]:
conv_base.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 326, 326, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 326, 326, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 326, 326, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 163, 163, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 163, 163, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 163, 163, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 81, 81, 128)       0     

In [4]:
plot_model(conv_base, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [137]:
original_dataset_dir_rust1 = 'Image Classification/Stage 1'
original_dataset_dir_rust2 = 'Image Classification/Stage 2'
original_dataset_dir_rust3 = 'Image Classification/Stage 3'
original_dataset_dir_rust4 = 'Image Classification/Stage 4'
original_dataset_dir_norust = 'norust'

#dir to store images
store_directory = 'ttv_images'
train_dir = os.path.join(store_directory, 'train')
validation_dir = os.path.join(store_directory, 'validation')
test_dir = os.path.join(store_directory, 'test')

rust1_train = os.path.join(train_dir, 'Rust (Stage 1)')
rust2_train = os.path.join(train_dir, 'Rust (Stage 2)')
rust3_train = os.path.join(train_dir, 'Rust (Stage 3)')
rust4_train = os.path.join(train_dir, 'Rust (Stage 4)')
norust_train = os.path.join(train_dir, 'No Rust')

norust_validation = os.path.join(validation_dir, 'No Rust ')
rust1_validation = os.path.join(validation_dir, 'Rust (Stage 1)')
rust2_validation = os.path.join(validation_dir, 'Rust (Stage 2)')
rust3_validation = os.path.join(validation_dir, 'Rust (Stage 3)')
rust4_validation = os.path.join(validation_dir, 'Rust (Stage 4)')

rust1_test = os.path.join(test_dir, 'Rust (Stage 1)')
rust2_test = os.path.join(test_dir, 'Rust (Stage 2)')
rust3_test = os.path.join(test_dir, 'Rust (Stage 3)')
rust4_test = os.path.join(test_dir, 'Rust (Stage 4)')
norust_test = os.path.join(test_dir, 'No Rust')



In [12]:

##runs a single time
# os.mkdir(store_directory)
# os.mkdir(train_dir)
# os.mkdir(validation_dir)
# os.mkdir(test_dir)
# os.mkdir(rust_train)
os.mkdir(norust_train)
# os.mkdir(rust_validation)
os.mkdir(norust_validation)
# os.mkdir(rust_test)
os.mkdir(norust_test)
# os.mkdir(rust1_train)
# os.mkdir(rust2_train)
# os.mkdir(rust3_train)
# os.mkdir(rust4_train)

# os.mkdir(rust1_validation)
# os.mkdir(rust2_validation)
# os.mkdir(rust3_validation)
# os.mkdir(rust4_validation)

# os.mkdir(rust1_test)
# os.mkdir(rust2_test)
# os.mkdir(rust3_test)
# os.mkdir(rust4_test)



In [8]:
fnames =  os.listdir(original_dataset_dir_norust)
print(len(fnames))

78


In [13]:
# #copies first 75 images to train 
filenames =  os.listdir(original_dataset_dir_rust1)
for i in range(0, 20):
    src = os.path.join(original_dataset_dir_rust1, filenames[i])
    dst = os.path.join(rust1_train, filenames[i])
    shutil.copyfile(src, dst)

# #copies first 5 images to Validation 

for i in range(19, 24):
    src = os.path.join(original_dataset_dir_rust1, filenames[i])
    dst = os.path.join(rust1_validation, filenames[i])
    shutil.copyfile(src, dst)

#copies first 5 images to train 
for i in range(24, 27):
    src = os.path.join(original_dataset_dir_rust1, filenames[i])
    dst = os.path.join(rust1_test, filenames[i])
    shutil.copyfile(src, dst)

##Rust 2
filenames =  os.listdir(original_dataset_dir_rust2)
for i in range(0, 55):
    src = os.path.join(original_dataset_dir_rust2, filenames[i])
    dst = os.path.join(rust2_train, filenames[i])
    shutil.copyfile(src, dst)

# #copies first 5 images to Validation 

for i in range(56, 61):
    src = os.path.join(original_dataset_dir_rust2, filenames[i])
    dst = os.path.join(rust2_validation, filenames[i])
    shutil.copyfile(src, dst)

#copies first 5 images to train 
for i in range(61, 66):
    src = os.path.join(original_dataset_dir_rust2, filenames[i])
    dst = os.path.join(rust2_test, filenames[i])
    shutil.copyfile(src, dst)

############################################
#Rust 3
filenames =  os.listdir(original_dataset_dir_rust3)
for i in range(0, 34):
    src = os.path.join(original_dataset_dir_rust3, filenames[i])
    dst = os.path.join(rust3_train, filenames[i])
    shutil.copyfile(src, dst)

# #copies first 5 images to Validation 

for i in range(32, 37):
    src = os.path.join(original_dataset_dir_rust3, filenames[i])
    dst = os.path.join(rust3_validation, filenames[i])
    shutil.copyfile(src, dst)

#copies first 5 images to train 
for i in range(36, 41):
    src = os.path.join(original_dataset_dir_rust3, filenames[i])
    dst = os.path.join(rust3_test, filenames[i])
    shutil.copyfile(src, dst)


#####################
#Rust4
filenames =  os.listdir(original_dataset_dir_rust4)
for i in range(0, 14):
    src = os.path.join(original_dataset_dir_rust4, filenames[i])
    dst = os.path.join(rust4_train, filenames[i])
    shutil.copyfile(src, dst)

# #copies first 5 images to Validation 

for i in range(13, 17):
    src = os.path.join(original_dataset_dir_rust4, filenames[i])
    dst = os.path.join(rust4_validation, filenames[i])
    shutil.copyfile(src, dst)

#copies first 5 images to train 
for i in range(17, 20):
    src = os.path.join(original_dataset_dir_rust4, filenames[i])
    dst = os.path.join(rust4_test, filenames[i])
    shutil.copyfile(src, dst)
#####
#No Rust
filenames =  os.listdir(original_dataset_dir_norust)
for i in range(0, 67):
    src = os.path.join(original_dataset_dir_norust, filenames[i])
    dst = os.path.join(norust_train, filenames[i])
    shutil.copyfile(src, dst)

for i in range(64, 70):
    src = os.path.join(original_dataset_dir_norust, filenames[i])
    dst = os.path.join(norust_validation, filenames[i])
    shutil.copyfile(src, dst)

for i in range(70, 78):
    src = os.path.join(original_dataset_dir_norust, filenames[i])
    dst = os.path.join(norust_test, filenames[i])
    shutil.copyfile(src, dst)


In [146]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='swish'))
model.add(layers.Dense(5, activation='softmax'))





In [148]:
model.summary()
conv_base.trainable = False 

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 10, 10, 512)       20024384  
_________________________________________________________________
flatten_18 (Flatten)         (None, 51200)             0         
_________________________________________________________________
dense_55 (Dense)             (None, 256)               13107456  
_________________________________________________________________
dense_56 (Dense)             (None, 5)                 1285      
Total params: 33,133,125
Trainable params: 13,108,741
Non-trainable params: 20,024,384
_________________________________________________________________


In [144]:
base_dir = 'ttv_images'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(326, 326),
        batch_size=6,
        shuffle=True,
        class_mode='categorical')


validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(326, 326),
        batch_size=4,
        shuffle=True,
        class_mode='categorical')

Found 196 images belonging to 5 classes.
Found 24 images belonging to 5 classes.


In [149]:
from time import time
from keras.callbacks import TensorBoard

# tensorboard = keras.callbacks.TensorBoard(log_dir='/output/{}'.format(time()))

# model.compile(loss='binary_crossentropy',optimizer=optimizers.RMSprop(lr=2e-5),metrics=['acc'])

model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])


history = model.fit_generator(train_generator,steps_per_epoch=10,epochs=17,validation_data=validation_generator,validation_steps=4,verbose=2)

Epoch 1/17
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @t

In [122]:
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(350, 350),
        batch_size=4,
        shuffle=False, 
        class_mode='categorical')

test_loss, test_acc = model.evaluate_generator(test_generator, steps=10)
print('test acc:', test_acc)

Found 27 images belonging to 5 classes.
test acc: 0.5925925970077515


In [152]:
test_data_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(326, 326),
        batch_size=3,
        shuffle=False, # v imp : do not shuffle in case of test data, when measuring precision and recall
        class_mode='categorical')

test_steps_per_epoch = np.math.ceil(test_data_generator.samples / test_data_generator.batch_size)

predictions = model.predict_generator(test_data_generator, steps=test_steps_per_epoch)

val_preds = np.argmax(predictions, axis=-1)
# for i in range(predictions.shape[0]):
#     if predictions[i]>0.5:
#         val_preds[i] = 1
#     else:
#         val_preds[i] = 0
val_trues = test_data_generator.classes

labels = test_data_generator.class_indices.keys()
cm = confusion_matrix(val_trues, val_preds)
print(cm)
report = classification_report(val_trues, val_preds, target_names=labels)
print(report) 
print(val_preds)

Found 27 images belonging to 5 classes.
[[10  0  0  0  0]
 [ 1  0  3  0  0]
 [ 2  0  1  2  0]
 [ 1  0  2  2  0]
 [ 0  0  0  3  0]]
              precision    recall  f1-score   support

      norust       0.71      1.00      0.83        10
       rust1       0.00      0.00      0.00         4
       rust2       0.17      0.20      0.18         5
       rust3       0.29      0.40      0.33         5
       rust4       0.00      0.00      0.00         3

    accuracy                           0.48        27
   macro avg       0.23      0.32      0.27        27
weighted avg       0.35      0.48      0.40        27

[0 0 0 0 0 0 0 0 0 0 2 2 0 2 0 0 3 2 3 3 2 3 0 2 3 3 3]


In [153]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('signature_function', 'signature_key'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('signature_function', 'signature_key'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('signature_function', 'signature_key'), but source function had ()
To silence this warning

In [154]:
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)